设置仿真参数

In [63]:
import random
import simpy

RANDOM_SEED = 42
REQ_TOTAL = 100 # 总请求数
ARRIVAL_RATE =  15.0 # 请求到达率 (每毫秒来多少个请求)
SERVICE_RATE =  10.0 # 服务速率   (每毫秒做多少个请求)
OVER_TIME    = 100.0 # 请求超时   (毫秒)

latency = [] # 采集请求延迟


初始化仿真对象

In [64]:
def workload(env, number, interval, server):
    """基于所设均值泊松分布生成请求"""
    for i in range(number):
        c = req_handler(env, 'Request%03d' % i, server, service_rate=SERVICE_RATE)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

def req_handler(env, name, server, service_rate):
    """请求处理过程"""
    arrive = env.now
    print('%7.4f %s: Incoming request' % (arrive, name))

    with server.request() as req:
        # ot = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        ot = random.uniform(1, 3)
        # 等待服务或超时
        results = yield req | env.timeout(OVER_TIME)

        wait = env.now - arrive

        if req in results:
            # We got to the server
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / service_rate)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))

            # 请求正常完成，采集延迟
            latency.append(env.now - arrive)

        else:
            # 请求超时
            print('%7.4f %s: Overtime after %6.3f' % (env.now, name, wait))

# 仿真环境准备
random.seed(RANDOM_SEED)
env = simpy.Environment()

开始仿真

In [65]:
server = simpy.Resource(env, capacity=1)
env.process(workload(env, REQ_TOTAL, ARRIVAL_RATE, server))
env.run()

 0.0000 Request000: Incoming request
 0.0000 Request000: Waited  0.000
 3.2162 Request000: Finished
15.3009 Request001: Incoming request
15.3009 Request001: Waited  0.000
19.0897 Request002: Incoming request
26.5926 Request001: Finished
26.5926 Request002: Waited  7.503
32.0731 Request002: Finished
52.4990 Request003: Incoming request
52.4990 Request003: Waited  0.000
52.9528 Request004: Incoming request
53.3562 Request005: Incoming request
59.5382 Request003: Finished
59.5382 Request004: Waited  6.585
62.0284 Request004: Finished
62.0284 Request005: Waited  8.672
69.0986 Request006: Incoming request
70.9265 Request005: Finished
70.9265 Request006: Waited  1.828
87.3162 Request006: Finished
93.9646 Request007: Incoming request
93.9646 Request007: Waited  0.000
95.6545 Request007: Finished
111.9315 Request008: Incoming request
111.9315 Request008: Waited  0.000
112.9048 Request008: Finished
159.2043 Request009: Incoming request
159.2043 Request009: Waited  0.000
160.7301 Request010: Inc

记录延迟数据

In [66]:
with open("latency.csv", "w+") as tracefile:
    tracefile.write("# latency\n")
    tracefile.writelines([str(l) + '\n' for l in latency])